In [16]:
from dataclasses import dataclass

import torch
from ultralytics import YOLO
from ultralytics.engine.results import Results
from ultralytics.engine.results import Boxes
from typing import List
from PIL import Image


In [17]:
# Load the model from weights

model = YOLO('best_so_far.pt')

In [18]:
# Inference on a single image

test_images = [
    'test_images/footprint_1.jpeg',
]

results = model.predict(
    source=test_images[0],
    conf=0.25,  # Confidence threshold
    iou=0.45,   # IoU threshold
    save_conf=True,  # Save confidence scores
)

print(results)


image 1/1 /home/spokay/projects/pycharms/Wildlens_Model/notebooks/yolo/test_images/footprint_1.jpeg: 352x640 2 items, 4.2ms
Speed: 0.9ms preprocess, 4.2ms inference, 0.7ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs/detect/predict2
1 label saved to runs/detect/predict2/labels
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'item'}
obb: None
orig_img: array([[[ 85,  81,  92],
        [ 26,  25,  35],
        [ 33,  32,  42],
        ...,
        [112, 109, 111],
        [144, 140, 145],
        [153, 149, 154]],

       [[134, 130, 141],
        [ 85,  84,  94],
        [ 60,  61,  71],
        ...,
        [111, 106, 108],
        [165, 160, 162],
        [182, 177, 179]],

       [[129, 128, 138],
        [101, 102, 112],
        [ 91,  92, 102],
        ...,
        [168, 162, 163],
        [164, 158, 159],
        [122, 116, 117]],

       ...,

       [[ 72,

In [19]:
# check bounding boxes
print(results[0].boxes)
print(results[0].boxes.shape)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0., 0.], device='cuda:0')
conf: tensor([0.7834, 0.5766], device='cuda:0')
data: tensor([[3.5629e+02, 3.1329e+01, 6.0497e+02, 2.5727e+02, 7.8338e-01, 0.0000e+00],
        [9.2517e+01, 8.2934e+01, 2.9249e+02, 2.7693e+02, 5.7661e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (386, 736)
shape: torch.Size([2, 6])
xywh: tensor([[480.6293, 144.3003, 248.6881, 225.9423],
        [192.5028, 179.9319, 199.9716, 193.9949]], device='cuda:0')
xywhn: tensor([[0.6530, 0.3738, 0.3379, 0.5853],
        [0.2616, 0.4661, 0.2717, 0.5026]], device='cuda:0')
xyxy: tensor([[356.2852,  31.3291, 604.9734, 257.2715],
        [ 92.5170,  82.9344, 292.4886, 276.9293]], device='cuda:0')
xyxyn: tensor([[0.4841, 0.0812, 0.8220, 0.6665],
        [0.1257, 0.2149, 0.3974, 0.7174]], device='cuda:0')
torch.Size([2, 6])


In [20]:
# check confidence scores
print(results[0].boxes.conf)

tensor([0.7834, 0.5766], device='cuda:0')


In [ ]:
# Calculate areas of bounding boxes
normalize_bbox = lambda bbox: [float(coord) for coord in bbox]

def calculer_areas_bbox_normalised(x1: float, y1: float, x2: float, y2: float):
    largeur = abs(x2 - x1)
    hauteur = abs(y2 - y1)
    return largeur * hauteur

def get_areas_from_bboxes(bbox: Boxes) -> List[float]:
    aires = []
    for bbox in bbox.xyxy:
        # Calculate area
        x1, y1, x2, y2 = normalize_bbox(bbox)
        aire = calculer_areas_bbox_normalised(x1, y1, x2, y2)
        aires.append(aire)

    return aires

In [ ]:
# Get an average of the areas and the confidence scores
@dataclass
class BBoxWithScore:
    bbox: List[float]
    score: float

def get_scrores_from_areas_and_confidence(areas: List[float], confidences: List[float]) -> List[float]:
    if len(areas) != len(confidences):
        raise ValueError("Areas and confidences must have the same length.")
    length = len(areas)
    scores = []
    for area, confidence in zip(areas, confidences):
        score = area * confidence / length if length > 0 else 0.0
        scores.append(score)

    return scores


def get_average_area_and_confidence(bboxes: Boxes) -> List[BBoxWithScore] | None:
    if bboxes is None or len(bboxes.xyxy) == 0:
        return None
    areas = get_areas_from_bboxes(bboxes)

    confidences = bboxes.conf.tolist()
    scores = get_scrores_from_areas_and_confidence(areas, confidences)
    bboxes_with_scores = [
        BBoxWithScore(bbox=bbox.tolist(), score=score)
        for bbox, score in zip(bboxes.xyxy, scores)
    ]

    return bboxes_with_scores

In [ ]:
# Get the best bounding box with the highest score
def get_best_bbox(bboxes_with_scores: List[BBoxWithScore]) -> BBoxWithScore | None:
    if not bboxes_with_scores:
        return None
    best_bbox = max(bboxes_with_scores, key=lambda x: x.score)
    return best_bbox

In [ ]:
# Check the BBOX results by cropping the images

def get_bboxes_from_results(result: Results) -> List[Boxes]:
    return result.boxes

def crop_images_from_results(results: List[Results], image_paths: List[str]) -> List[Image]:
    cropped_images = []
    for result, image_path in zip(results, image_paths):
        img = Image.open(image_path)
        bboxes = get_bboxes_from_results(result)
        for bbox in bboxes.xyxy:  # xyxy format
            x1, y1, x2, y2 = map(int, bbox[:4])  # Convert to int
            cropped_image = img.crop((x1, y1, x2, y2))  # Crop the image
            cropped_images.append(cropped_image)  # Append cropped image to list

    return cropped_images


test_cropped_images = crop_images_from_results(results, test_images)

# Display cropped images
for i, cropped_image in enumerate(test_cropped_images):
    cropped_image.show(title=f"Cropped Image {i+1}")
    # Optionally save the cropped images
    cropped_image.save(f"cropped_image_{i+1}.jpg")